# Análise de Dados de vendas

Colunas:

    home: indica se o usuário acessou ou não a página inicial.
    how_it_works: indica se o usuário acessou ou não a página de “Como funciona a Empresa”.
    contact: indica se o usuário acessou ou não a página de contato da empresa.
    bought: indica se o usuário efetuou uma compra ou não.

- Indetifique e extraia informações desses dados em formas de gráficos

* BONUS: faça uso de IA para gerar previsões de vendas

In [1]:
#Instalando pandas

%pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [2]:
#Importando base de dados
from IPython.display import display, HTML
import pandas as pd

df = pd.read_csv('tracking.csv')

# display(HTML(df.to_html())) #mostrar o dataframe completo
display(df) #mostrar dataframe simplificado

,home,how_it_works,contact,bought
0,1,1,0,0
1,1,1,0,0
2,1,1,0,0
3,1,1,0,0
4,1,1,0,0
...,...,...,...,...
95,0,0,0,0
96,1,1,1,1
97,0,0,1,0
98,0,1,0,0


In [3]:
#instalando o plotly (biblioteca de gráficos)

%pip install plotly

Note: you may need to restart the kernel to use updated packages.


In [4]:
#importando o plotly

import plotly.express as px

In [5]:
#Mapa de Calor

fig = px.imshow(df.corr(), labels=dict(y='Página acessada',
                                       color='Efetividade'),
               color_continuous_scale=px.colors.sequential.Greens,
               text_auto=True)

fig.update_layout(title={
    'text' : 'CORRELAÇÃO ENTRE VENDAS E PÁGINAS ACESSADAS',
    'x' : 0.5
})

fig.show()

In [6]:
#Gráfico de Histograma

columns = ['home', 'how_it_works', 'contact']
for column in columns:
    fig = px.histogram(df, x=column, color='bought', text_auto=True,
                      labels={'bought':'Compraram',
                             'home':'Página Inicial',
                             'how_it_works':'Página de Como Funciona a Empresa',
                             'contact':'Página de Contatos'},
                      color_discrete_sequence=['darkred','navy'])
    
    newnames = {'0':'Não', '1': 'Sim'}
    fig.for_each_trace(lambda t: t.update(name = newnames[t.name], 
                                          legendgroup = newnames[t.name], 
                                          hovertemplate = t.hovertemplate.replace(t.name, newnames[t.name])
                                         ))
    
    fig.update_layout(title={
    'text' : 'RELAÇÃO ENTRE ACESSOS E VENDAS <br><sup>OBS: 0 = Não acessou / 1 = Acessou</sup>',
    'x' : 0.5
    })
    fig.update_layout(bargap=0.2)
    fig.show()

In [7]:
#Gráficos de Barras

data = df[df['bought']==1]
values = data.sum()
index = [['home',values['home']],
         ['how_it_works',values['how_it_works']],
         ['contact',values['contact']]]

df_aux = pd.DataFrame(index, columns=['pagina', 'qtd'])

fig = px.bar(df_aux, x='pagina', y='qtd',
            labels={
                'pagina':'Páginas Acessadas',
                'qtd':'Quantidade de Acessos',
            },
            color_discrete_sequence=px.colors.qualitative.T10,
            text_auto=True)

fig.update_layout(title={
    'text' : 'PÁGINAS QUE MAIS GERARAM VENDAS',
    'x' : 0.5})

fig.show()

## INTELIGÊNCIA ARTIFICIAL


Nessa parte irei demonstrar um exemplo de previsão de vendas analisando as páginas acessadas

In [8]:
#instalando sklearn, principal lib de IA do python
%pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [9]:
"""
Dividindo Y e X

Sendo que:
    Y é quem eu quero prever
    X são os valores a serem analisados
"""

y = df['bought']
x = df[['home', 'how_it_works', 'contact']]


x = x.values #apenas colocado para se retirar o aviso no final

In [10]:
#Importando a IA
from sklearn.ensemble import RandomForestRegressor

In [11]:
#Dividindo a base de treino e de teste

#Para isso tbm preciso do import de um modulo do sklearn o train_test_split
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y)

In [12]:
#Criando o modelo da IA
randomForest_model = RandomForestRegressor()

In [13]:
#Treinando a IA
randomForest_model.fit(x_train, y_train)

RandomForestRegressor()

In [14]:
#Testando a IA

predict_randomForest = randomForest_model.predict(x_test)

In [15]:
#Testando a IA com dados do usuário e verificando a acertibilidade
#Para verificar a acertibilidade precisamos do r²
from sklearn.metrics import r2_score


print('*'*50)
print('Responda 1 para SIM e 0 para NÃO'.center(50))
print('*'*50)

home = int(input("\nAcessou a Página Inicial? "))
how_it_works = int(input("Acessou a Página de Como a Empresa Funciona? "))
contact = int(input("Acessou a Página de Contatos? "))

predict = randomForest_model.predict([[home, how_it_works, contact]])

print(f'\nA probabilidade de gerar venda é de: {float(predict):,.4%}')
print(f'A IA possui acertibilidade de {r2_score(y_test, predict_randomForest):,.4%}')

**************************************************
         Responda 1 para SIM e 0 para NÃO         
**************************************************

A probabilidade de gerar venda é de: 11.3534%
A IA possui acertibilidade de 98.6573%
